# Caffe Shrinking Tool(Coarse-Prune) For VGG-DictNet

## 1. Import Library

In [ ]:
# Caffe Directory Setting
caffe_root = '../'

import sys 
import os
import numpy as np
import os.path as osp
import codecs

# Import Caffe Python Library
sys.path.append(caffe_root + 'python')
import caffe 

# Import Custom Library
sys.path.append("utils")
sys.path.append("utils/layers")
sys.path.append("utils/nets")
sys.path.append("utils/net_shrink")

from multilabel_datalayers import *
from pynet import *
import tools
from shrink_tools import *

## 2. Setup Parameters

In [1]:
# Model Setting
output_num = 88172
working_folder = "vgg_dictnet_shrink_%d_n/" % output_classes

# Training & Testing Path
vgg_data_path = "../vgg_data/"
train_file = vgg_data_path + "train.txt"
val_file = vgg_data_path + "val.txt"

# Hyper-Parameters
policy = "keep"
batch_sz = 256
train_epoches = 1
retrain = True
step_size = train_iteration // 1
input_fc = [4096, 4096, output_num]
output_fc = [3821, 3821, output_num]

# Caffe Mode
caffe.set_mode_gpu()
caffe.set_device(0)

## 3. Get VGG-dictnet Data & Model 

In [4]:
# Get VGG-DictNet Data
if os.path.isdir(caffe_root + "vgg_data"):
    print 'VGG Data found!'
else:
    print 'Downloading VGG Data (about 10G)'
    !wget http://www.robots.ox.ac.uk/~vgg/data/text/mjsynth.tar.gz -0 {caffe_root}/vgg_data.tar.gz
    !tar xvf {caffe_root}/vgg_data.tar.gz -C {caffe_root}/vgg_data
    
# Get VGG-DictNet Model
if os.path.isfile(caffe_root + 'models/vgg_dictnet_mtoc/dictnet_vgg_mtoc.caffemodel'):
    print 'VGG Models found.'
else:
    print 'Downloading pre-trained VGG models...'
    !wget https://www.dropbox.com/s/i4tu1rq7r021xkr/vgg_dictnet_mtoc.tar.gz
    ! mkdir  {caffe_root}models/vgg_dictnet_mtoc
    !tar xvf vgg_dictnet_mtoc.tar.gz -C {caffe_root}models/vgg_dictnet_mtoc
    ! rm vgg_dictnet_mtoc_10000.tar.gz
    
model_load = osp.join(caffe_root + 'models/vgg_dictnet_mtoc/dictnet_vgg_mtoc.caffemodel')
data_list = "../vgg_data/lexicon.txt"  

VGG Data found!


In [ ]:
def reTrainModel(solver, train_iteration, threshold):
    
    print "Train Iteration: %d" % train_iteration
    loss = []
    
    for i in range(train_iteration // 200+1):
        solver.step(200)
        loss += solver.net.blobs['loss'].data
        if len(loss) == 6:
            loss.pop(0)
        elif np.mean(loss) < threshold:
            break

## 4. Start Pruning

In [8]:
# If loss less than 0.3, than break the for-loop to save model
loss_threshold = 0.3

train_iteration =len(codecs.open(train_file, 'r', 'utf8').readlines()) * train_epoches / batch_sz

if osp.isdir(osp.join(working_folder, "models")) is False:
    os.makedirs(osp.join(working_folder, "models"))
    
!cp {model_load} {osp.join(working_folder, "models", "snap_fc1_0_fc2_0.caffemodel")}
    
pro = Create_prototxt(working_folder, policy, batch_sz, data_list, step_size, train_file, val_file)

solver = shrink_fc(pro, working_folder, input_fc, output_fc, step)
    
    
# Start Retrain
if retrain is True:
    solver = reTrainModel(solver_d, train_iteration, loss_threshold)

solver.net.save(osp.join(working_folder,"models", "new.caffemodel"))

/bin/sh: 1: Syntax error: "(" unexpected
vgg_dictnet_shrink_10000_n/prototxt/4000/solver_fc1_4000_fc2_4000.prototxt
vgg_dictnet_shrink_10000_n/prototxt/4000/trainnet_fc1_4000_fc2_4000.prototxt
vgg_dictnet_shrink_10000_n/prototxt/4000/valnet_fc1_4000_fc2_4000.prototxt
vgg_dictnet_shrink_10000_n/prototxt/4000/deploynet_fc1_4000_fc2_4000.prototxt
Train Iteration: 3233
New Model Name: snap_fc1_4000_fc2_0.caffemodel
---------------------------------------------------


/home/ubuntu/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
